In [1]:
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
import pandas as pd
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster

import utils
import matplotlib.pyplot as plt

from pandas import read_csv

['c:\\Users\\jz421\\Desktop\\GlobalLocal', 'C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\L

c:\Users\jz421\AppData\Local\anaconda3\envs\ieeg\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


make model rdms here (just do congruency and switch type for testing for now 5/4)

In [2]:
# Define the conditions
conditions = ["i25s25", "i25s75", "i75s25", "i75s75", "i25r25", "i25r75", "i75r25", "i75r75",
              "c25s25", "c25s75", "c75s25", "c75s75", "c25r25", "c25r75", "c75r25", "c75r75"]

# Create a list to extract the congruency ('c' or 'i') from each condition
congruency = [cond[0] for cond in conditions]

# Create a list to extract the switch type ('s' or 'r') from each condition
switch_types = [cond[-3] for cond in conditions]

# Number of conditions
n = len(conditions)

# Initialize the RDM with ones
model_congruency_RDM = np.ones((n, n))

# Initialize the RDM with ones
model_switchType_RDM = np.ones((n, n))

# Set entries to 0 where switch types match
for i in range(n):
    for j in range(n):
        if switch_types[i] == switch_types[j]:
            model_switchType_RDM[i, j] = 0
        if congruency[i] == congruency[j]:
            model_congruency_RDM[i,j] = 0

In [8]:
print(model_congruency_RDM)

[[0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [9]:
print(model_switchType_RDM)

[[0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1.]
 [0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1.]
 [0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1.]
 [0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1.]
 [1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0.]
 [1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0.]
 [1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0.]
 [1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1.]
 [0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1.]
 [0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1.]
 [0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1.]
 [1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0.]
 [1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0.]
 [1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0.]
 [1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0.]]


#### load in epochs data  
(get only acc trials, avg across epochs to make evoked using only acc trials, and then delete epochs from memory) 5/5

In [3]:
subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']

make or load subjects electrodes to rois dict (mapping from electrode names to roi labels)

In [4]:
# load in subjects electrodes to rois dict. If it doesn't already exist, make it and then load it.
filename = 'subjects_electrodestoROIs_dict.json'
subjects_electrodestoROIs_dict = utils.make_or_load_subjects_electrodes_to_rois_dict(filename, subjects)

Attempting to load the subjects' electrodes-to-ROIs dictionary...
Loaded data from subjects_electrodestoROIs_dict.json
Dictionary loaded successfully. Ready to proceed!


load accuracy arrays so we can filter by only accurate trials  

In [5]:
# this makes numpy arrays for each subject that are 0 or 1 for each trial based on accuracy
from makeRawBehavioralData import main
main()

# Directory where your .npy files are saved
npy_directory = r'C:\Users\jz421\Box\CoganLab\D_Data\GlobalLocal\accArrays'  # Replace with your directory path if you're not Jim

acc_array = utils.load_acc_arrays(npy_directory, skip_subjects=['D107'])

load in behavioral data and convert block types to congruency and switch proportions

In [6]:
combined_data = pd.read_csv(r'C:\Users\jz421\Box\CoganLab\D_Data\GlobalLocal\combinedData.csv')

# Apply the function to each row and create new columns
combined_data[['congruencyProportion', 'switchProportion']] = combined_data.apply(utils.map_block_type, axis=1)

load epochs and evoked for all 16 conditions

In [7]:
output_names = ["Stimulus_i25s25_fixationCrossBase_1sec_mirror_0to1Test", "Stimulus_i25s75_fixationCrossBase_1sec_mirror_0to1Test", 
                "Stimulus_i75s25_fixationCrossBase_1sec_mirror_0to1Test", "Stimulus_i75s75_fixationCrossBase_1sec_mirror_0to1Test", 
                "Stimulus_i25r25_fixationCrossBase_1sec_mirror_0to1Test", "Stimulus_i25r75_fixationCrossBase_1sec_mirror_0to1Test", 
                "Stimulus_i75r25_fixationCrossBase_1sec_mirror_0to1Test", "Stimulus_i75r75_fixationCrossBase_1sec_mirror_0to1Test", 
                "Stimulus_c25s25_fixationCrossBase_1sec_mirror_0to1Test", "Stimulus_c25s75_fixationCrossBase_1sec_mirror_0to1Test", 
                "Stimulus_c75s25_fixationCrossBase_1sec_mirror_0to1Test", "Stimulus_c75s75_fixationCrossBase_1sec_mirror_0to1Test",
                "Stimulus_c25r25_fixationCrossBase_1sec_mirror_0to1Test", "Stimulus_c25r75_fixationCrossBase_1sec_mirror_0to1Test",
                "Stimulus_c75r25_fixationCrossBase_1sec_mirror_0to1Test", "Stimulus_c75r75_fixationCrossBase_1sec_mirror_0to1Test",]
                
output_names_conditions = {
    "Stimulus_i25s25_fixationCrossBase_1sec_mirror_0to1Test": {
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    }
    # "Stimulus_i25s75_fixationCrossBase_1sec_mirror_0to1Test": {
    #     "congruency": "i",
    #     "congruencyProportion": "75%",
    #     "switchType": "s",
    #     "switchProportion": "75%"
    # },
    # "Stimulus_i75s25_fixationCrossBase_1sec_mirror_0to1Test": {
    #     "congruency": "i",
    #     "congruencyProportion": "25%",
    #     "switchType": "s",
    #     "switchProportion": "25%"
    # },
    # "Stimulus_i75s75_fixationCrossBase_1sec_mirror_0to1Test": {
    #     "congruency": "i",
    #     "congruencyProportion": "25%",
    #     "switchType": "s",
    #     "switchProportion": "75%"
    # },
    # "Stimulus_i25r25_fixationCrossBase_1sec_mirror_0to1Test": {
    #     "congruency": "i",
    #     "congruencyProportion": "75%",
    #     "switchType": "r",
    #     "switchProportion": "25%"
    # },
    # "Stimulus_i25r75_fixationCrossBase_1sec_mirror_0to1Test": {
    #     "congruency": "i",
    #     "congruencyProportion": "75%",
    #     "switchType": "r",
    #     "switchProportion": "75%"
    # },
    # "Stimulus_i75r25_fixationCrossBase_1sec_mirror_0to1Test": {
    #     "congruency": "i",
    #     "congruencyProportion": "25%",
    #     "switchType": "r",
    #     "switchProportion": "25%"
    # },
    # "Stimulus_i75r75_fixationCrossBase_1sec_mirror_0to1Test": {
    #     "congruency": "i",
    #     "congruencyProportion": "25%",
    #     "switchType": "r",
    #     "switchProportion": "75%"
    # },
    # "Stimulus_c25s25_fixationCrossBase_1sec_mirror_0to1Test": {
    #     "congruency": "c",
    #     "congruencyProportion": "75%",
    #     "switchType": "s",
    #     "switchProportion": "25%"
    # },
    # "Stimulus_c25s75_fixationCrossBase_1sec_mirror_0to1Test": {
    #     "congruency": "c",
    #     "congruencyProportion": "75%",
    #     "switchType": "s",
    #     "switchProportion": "75%"
    # },
    # "Stimulus_c75s25_fixationCrossBase_1sec_mirror_0to1Test": {
    #     "congruency": "c",
    #     "congruencyProportion": "25%",
    #     "switchType": "s",
    #     "switchProportion": "25%"
    # },
    # "Stimulus_c75s75_fixationCrossBase_1sec_mirror_0to1Test": {
    #     "congruency": "c",
    #     "congruencyProportion": "25%",
    #     "switchType": "s",
    #     "switchProportion": "75%"
    # },
    # "Stimulus_c25r25_fixationCrossBase_1sec_mirror_0to1Test": {
    #     "congruency": "c",
    #     "congruencyProportion": "75%",
    #     "switchType": "r",
    #     "switchProportion": "25%"
    # },
    # "Stimulus_c25r75_fixationCrossBase_1sec_mirror_0to1Test": {
    #     "congruency": "c",
    #     "congruencyProportion": "75%",
    #     "switchType": "r",
    #     "switchProportion": "75%"
    # },
    # "Stimulus_c75r25_fixationCrossBase_1sec_mirror_0to1Test": {
    #     "congruency": "c",
    #     "congruencyProportion": "25%",
    #     "switchType": "r",
    #     "switchProportion": "25%"
    # },
    # "Stimulus_c75r75_fixationCrossBase_1sec_mirror_0to1Test": {
    #     "congruency": "c",
    #     "congruencyProportion": "25%",
    #     "switchType": "r",
    #     "switchProportion": "75%"
    # },                                                            
}

task='GlobalLocal'

# Assuming 'combined_data' is your DataFrame and 'subjects' is your list of subject IDs
subjects_mne_objects = utils.create_subjects_mne_objects_dict(subjects, output_names_conditions, task="GlobalLocal", combined_data=combined_data, acc_array=acc_array)

Loading data for subject: D0057
  Loading output: Stimulus_i25s25_fixationCrossBase_1sec_mirror_0to1Test with conditions: {'congruency': 'i', 'congruencyProportion': '75%', 'switchType': 's', 'switchProportion': '25%'}
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_i25s25_fixationCrossBase_1sec_mirror_0to1Test_HG_ev1-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
7 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_i25s25_fixationCrossBase_1sec_mirror_0to1Test_HG_base-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...       0.00 ms
        0 CTF compensation matrices available
Not setting metadata
448 matching events found
No baseline correction applied
0 projectio

load stimulus significant channels

In [8]:
sig_chans_per_subject = utils.get_sig_chans_per_subject(subjects, task='GlobalLocal', LAB_root=None)

Loaded significant channels for subject D0057
Loaded significant channels for subject D0059
Loaded significant channels for subject D0063
Loaded significant channels for subject D0065
Loaded significant channels for subject D0069
Loaded significant channels for subject D0071
Loaded significant channels for subject D0077
Loaded significant channels for subject D0090
Loaded significant channels for subject D0094
Loaded significant channels for subject D0100
Loaded significant channels for subject D0102
Loaded significant channels for subject D0103


get electrodes for each roi (definitions based on destrieux atlas)

In [9]:
rois_dict = {
    # 'dlpfc': ["G_front_middle", "G_front_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
    # 'acc': ["G_and_S_cingul-Ant", "G_and_S_cingul-Mid-Ant"],
    # 'parietal': ["G_parietal_sup", "S_intrapariet_and_P_trans", "G_pariet_inf-Angular", "G_pariet_inf-Supramar"],
    'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"]
}

rois = list(rois_dict.keys())
electrodes_per_subject_roi, sig_electrodes_per_subject_roi = utils.make_sig_electrodes_per_subject_and_roi_dict(rois_dict, subjects_electrodestoROIs_dict, sig_chans_per_subject)

For subject D0057, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['RAI6', 'RAI12', 'RAI13', 'RAI15', 'RAI16', 'RPI14', 'RAMF10', 'RAMF11', 'RAMF12', 'RAIF11', 'RAIF12', 'RAIF13', 'RAIF14']
For subject D0059, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LMMF9', 'LMMF11', 'LMMF12', 'LPSF16']
For subject D0063, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LOF16', 'LASF10', 'LASF14', 'LASF15', 'LASF16', 'LM

get electrode counts for each roi (just for fun)

In [10]:
total_electrodes_info = utils.calculate_total_electrodes(sig_electrodes_per_subject_roi, electrodes_per_subject_roi)
for roi, counts in total_electrodes_info.items():
    print(f"Total number of significant {roi} electrodes across all subjects:", counts['total_significant_electrodes'])
    print(f"Total number of {roi} electrodes across all subjects:", counts['total_electrodes'])

Total number of significant lpfc electrodes across all subjects: 44
Total number of lpfc electrodes across all subjects: 105


get trial averaged data

In [12]:
# Define time indices just for the function to work. Don't actually use time averaged data.
time_indices = {
    'firstHalfSecond': (2048, 3072),
    'secondHalfSecond': (3072, 4096),
    'fullSecond': (2048, 4096)
}
output_names = ["Stimulus_i25s25_fixationCrossBase_1sec_mirror_0to1Test"]
# Process the data
data_trialAvg_lists, data_trialStd_lists, _, overall_electrode_mapping, electrode_mapping_per_roi = utils.process_data_for_roi(
    subjects_mne_objects, output_names, rois, subjects, sig_electrodes_per_subject_roi, time_indices)

# need to figure out how to use the overall_electrode_mapping to get the electrode names back. Apparently I do this in the ANOVA in roi analysis. 5/5.

# Concatenate the data
concatenated_trialAvg_data = utils.concatenate_data(data_trialAvg_lists, rois, output_names)

Subject: D0057, ROI: lpfc, Num of Sig Electrodes: 2
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


Subject: D0059, ROI: lpfc, Num of Sig Electrodes: 4
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


c:\Users\jz421\Desktop\GlobalLocal\utils.py:269: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
c:\Users\jz421\Desktop\GlobalLocal\utils.py:270: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epochs_data = epochs.get_data().copy()
c:\Users\jz421\Desktop\GlobalLocal\utils.py:269: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
c:\Users\jz421\Desktop\GlobalLocal\utils.py:270: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epochs_data = epochs.get_data().copy(

Subject: D0063, ROI: lpfc, Num of Sig Electrodes: 8
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Subject: D0065, ROI: lpfc, Num of Sig Electrodes: 1
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


c:\Users\jz421\Desktop\GlobalLocal\utils.py:563: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  epochs = subjects_mne_objects[sub][output_name]['HG_ev1_rescaled'].copy().pick_channels(sig_electrodes)
c:\Users\jz421\Desktop\GlobalLocal\utils.py:269: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
c:\Users\jz421\Desktop\GlobalLocal\utils.py:270: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epochs_data = epochs.get_data().copy()
c:\Users\jz421\Desktop\GlobalLocal\utils.py:283: RuntimeWarning: Mean o

...

c:\Users\jz421\Desktop\GlobalLocal\utils.py:563: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  epochs = subjects_mne_objects[sub][output_name]['HG_ev1_rescaled'].copy().pick_channels(sig_electrodes)
c:\Users\jz421\Desktop\GlobalLocal\utils.py:269: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
c:\Users\jz421\Desktop\GlobalLocal\utils.py:270: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epochs_data = epochs.get_data().copy()
c:\Users\jz421\Desktop\GlobalLocal\utils.py:269: FutureWarning: The cur

Subject: D0094, ROI: lpfc, Num of Sig Electrodes: 10
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


c:\Users\jz421\Desktop\GlobalLocal\utils.py:269: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
c:\Users\jz421\Desktop\GlobalLocal\utils.py:270: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epochs_data = epochs.get_data().copy()
c:\Users\jz421\Desktop\GlobalLocal\utils.py:269: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
c:\Users\jz421\Desktop\GlobalLocal\utils.py:270: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epochs_data = epochs.get_data().copy(

Subject: D0100, ROI: lpfc, Num of Sig Electrodes: 0
Subject: D0102, ROI: lpfc, Num of Sig Electrodes: 3
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Subject: D0103, ROI: lpfc, Num of Sig Electrodes: 6
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


c:\Users\jz421\Desktop\GlobalLocal\utils.py:269: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
c:\Users\jz421\Desktop\GlobalLocal\utils.py:270: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epochs_data = epochs.get_data().copy()
c:\Users\jz421\Desktop\GlobalLocal\utils.py:269: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
c:\Users\jz421\Desktop\GlobalLocal\utils.py:270: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epochs_data = epochs.get_data().copy(

In [15]:
concatenated_trialAvg_data['Stimulus_i25s25_fixationCrossBase_1sec_mirror_0to1Test']['lpfc'].shape

(44, 5121)

In [16]:
concatenated_trialAvg_data

{'Stimulus_i25s25_fixationCrossBase_1sec_mirror_0to1Test': {'lpfc': array([[-0.27893369, -0.28428207, -0.2898255 , ...,  0.18689985,
           0.18496857,  0.18284192],
         [ 0.13412701,  0.13505303,  0.13582653, ...,  0.07705337,
           0.06740816,  0.05776003],
         [-0.02884331, -0.02668281, -0.02448124, ...,  1.00115065,
           1.00043964,  0.99929257],
         ...,
         [-0.64864258, -0.64674573, -0.64495685, ..., -0.33375768,
          -0.33872441, -0.34387202],
         [-0.26504986, -0.27681604, -0.28894591, ..., -0.49723368,
          -0.49161252, -0.48609994],
         [ 0.06897462,  0.07349734,  0.07797647, ...,  0.44334181,
           0.43653923,  0.42953495]])}}